In [1]:
import pandas as pd
import cooler
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import numpy as np
import seaborn as sns
import pandas as pd
%matplotlib inline

In [2]:
# Define chromosomes
c = cooler.Cooler('/data2/josh/stan/merge_res200.cool')
chrs = c.chroms()[:]
chrs['name'] = chrs['name'].str.replace('chr', '')
chrs.head()

,name,length
0,1,195471971
1,2,182113224
2,3,160039680
3,4,156508116
4,5,151834684


In [27]:
# H3K36ME3
peaks = pd.read_csv('~/mlep/chipseq/macs14_peaks.bed', delimiter='\t', header=None)

FileNotFoundError: File b'/home/josh/mlep/chipseq/macs14_peaks.bed' does not exist

In [4]:
peaks.columns = ['chr', 'start', 'end', 'id', 'strength']
peaks.head()

NameError: name 'peaks' is not defined

In [6]:
peaks_sorted = peaks.sort_values('strength', ascending=False)

In [3]:
# create df with bins
bins = c.bins()[['chrom', 'start', 'end']]
bins[977359:977365]

,chrom,start,end
977359,chr1,195471800,195471971
977360,chr2,0,200
977361,chr2,200,400
977362,chr2,400,600
977363,chr2,600,800
977364,chr2,800,1000


In [4]:
# create df with pixels
pix = c.pixels()
p = pix[:]
p.head()
# p.loc[0,'bin1_id']

,bin1_id,bin2_id,count
0,15000,15002,1
1,15000,15003,2
2,15000,15006,1
3,15000,15007,1
4,15000,15045,1


In [8]:
p_strong = p[p['count'] >= 5]

In [9]:
poi = p_strong[(p_strong['bin2_id'] - p_strong['bin1_id']) >= 25]
poi.head(2)

,bin1_id,bin2_id,count
704,15017,399915,5
185559,18361,18389,5


In [10]:
len(poi)

350325

In [11]:
poi.to_csv('/data2/josh/contacts/all_strong5_pixels.csv',sep=',',index=True, header=True)

In [25]:
# pix_pos = c.pixels(join=True)[:] # strange how long this takes relative to pix instantiation
# pix_pos.head()

# Reformatting 
# pix_pos['chrom1'] = int(pix_pos['chrom1'][-1])

In [13]:
# add position columns
poi['start1'] = pix['bin1_id'].multiply(200)

AttributeError: 'RangeSelector1D' object has no attribute 'multiply'

In [ ]:
def overlapping(left,right):
    overlap_count =  min(left[1], right[1]) - max(left[0], right[0]) # +1 because bordering doesn't count as intersecting
    print(overlap_count)
    if overlap_count > 0:
        return True
    return False

In [ ]:
pix_head = pix_pos.head()

pix_pos['chip1'] = ''
pix_pos['chip2'] = ''

# pix_head[0,'chip1'] = overlapping((start1, end1), ()

In [ ]:
# change into list comprehensions to speed up

for idx, row in pix_pos.iterrows():
    
    # How currently written, chip peak must totally encapsulate 200 bp of contact
    chip1 = peaks[(peaks['chr'] == row['chrom1'][-1]) 
                                       & (peaks['start'] < row['start1']) 
                                       & (peaks['end'] > row['end1'])]
    chip2 = peaks[(peaks['chr'] == row['chrom1'][-1]) 
                                       & (peaks['start'] < row['start2']) 
                                       & (peaks['end'] > row['end2'])]
    
    if chip1.empty:
        pix_pos.loc[idx, "chip1"] = False
    else: 
        pix_pos.loc[idx, "chip1"] = True
    if chip2.empty:
        pix_pos.loc[idx, "chip2"] = False  
    else: 
        pix_pos.loc[idx, "chip2"] = True
        

pix_pos.head()

In [ ]:
# pix_pos[pix_pos['chip2'] == True]